In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/nkm/Downloads/Task 3 and 4_Loan_Data.csv")

df.head()


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [2]:
print(df.columns)


Index(['customer_id', 'credit_lines_outstanding', 'loan_amt_outstanding',
       'total_debt_outstanding', 'income', 'years_employed', 'fico_score',
       'default'],
      dtype='object')


In [3]:
df.isnull().sum()


customer_id                 0
credit_lines_outstanding    0
loan_amt_outstanding        0
total_debt_outstanding      0
income                      0
years_employed              0
fico_score                  0
default                     0
dtype: int64

In [4]:
# Select features (X) and target (Y)
X = df[
    [
        "credit_lines_outstanding",
        "loan_amt_outstanding",
        "total_debt_outstanding",
        "income",
        "years_employed",
        "fico_score"
    ]
]

Y = df["default"]

# Check shapes
print("X shape:", X.shape)
print("Y shape:", Y.shape)


X shape: (10000, 6)
Y shape: (10000,)


In [6]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.1 MB 1.8 MB/s eta 0:00:05
   ------ --------------------------------- 1.3/8.1 MB 2.4 MB/s eta 0:00:03
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.6/8.1 MB 2.2 MB/s eta 0:00:04
   --------- ------------------------------ 1.8


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

print("Training set:", X_train.shape, Y_train.shape)
print("Test set:", X_test.shape, Y_test.shape)


Training set: (8000, 6) (8000,)
Test set: (2000, 6) (2000,)


In [8]:
from sklearn.linear_model import LogisticRegression

# Create model
pd_model = LogisticRegression(max_iter=1000)

# Train model
pd_model.fit(X_train, Y_train)

print("PD Model Training Complete")


PD Model Training Complete


In [9]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict on test data
Y_pred = pd_model.predict(X_test)

# Accuracy
print("Model Accuracy:", round(accuracy_score(Y_test, Y_pred), 4))

# Detailed report
print("\nClassification Report:\n", classification_report(Y_test, Y_pred))

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, Y_pred))


Model Accuracy: 0.997

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1652
           1       1.00      0.98      0.99       348

    accuracy                           1.00      2000
   macro avg       1.00      0.99      0.99      2000
weighted avg       1.00      1.00      1.00      2000


Confusion Matrix:
 [[1652    0]
 [   6  342]]


In [ ]:
# Metric	Meaning
#1652 (TN)	Safe customers correctly approved
#0 (FP)	Risky customers approved = ZERO (This is EXCELLENT)
#6 (FN)	Safe customers rejected (small business loss)
#342 (TP)	Risky customers correctly rejected
# This model is very conservative — it prevents bad loans almost perfectly,
# at the cost of rejecting a few good customers.”

In [10]:
# Now we use your model like a loan officer system.
import pandas as pd

# Example new customer
new_customer = pd.DataFrame([{
    "credit_lines_outstanding": 3,
    "loan_amt_outstanding": 5000,
    "total_debt_outstanding": 12000,
    "income": 45000,
    "years_employed": 4,
    "fico_score": 680
}])

# Predict probability of default
pd_value = pd_model.predict_proba(new_customer)[0][1]

print("Probability of Default (PD):", round(pd_value, 4))



Probability of Default (PD): 0.872


In [ ]:
# Banks often use:

# PD < 0.30 → Approve
# PD 0.30–0.60 → Manual Review
# PD > 0.60 → Reject

# A PD = 0.872 (87.2%) means your model says:
# “This customer is VERY likely to default” → REJECT

In [ ]:
# Banks don’t just ask:

“Will this customer default?”

They ask:

“How much MONEY will we lose if they default?”

🔥 Expected Loss Formula
EL = PD × LGD × EAD


Where:

Term	Meaning
PD  	Probability of Default (from your model)
LGD	    Loss Given Default (percentage of loan lost)
EAD	    Exposure at Default (loan amount at risk)

In [ ]:
# Define LGD & EAD Rules

We’ll use realistic banking assumptions:

LGD = 0.45 → Bank recovers ~55% via collections/legal

EAD = loan_amt_outstanding

In [11]:
# Expected Loss Function
def calculate_expected_loss(pd, loan_amount, lgd=0.45):
    return round(pd * lgd * loan_amount, 2)


In [13]:
# Apply It to Your Customer
loan_amount = 5000  # same as new customer loan

el = calculate_expected_loss(pd_value, loan_amount)

print("Expected Loss (EL): $", el)


Expected Loss (EL): $ 1961.92


In [ ]:
# If we give this customer a $5,000 loan, we expect to lose about $1,962 on average.”
# That’s CFO-level decision logic.